In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from sklearn import datasets 
data = datasets.load_iris() 
#create a DataFrame 
df = pd.DataFrame(data.data, columns=data.feature_names) 
df['Target'] = pd.DataFrame(data.target)
print(df.shape)
df.head()

In [ ]:
#visualisasi hasil ConvexHull
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull
plt.figure(figsize = (10, 6))
colors = ['b','r','g']
plt.title('Petal Width vs Petal Length')
plt.xlabel(data.feature_names[0])
plt.ylabel(data.feature_names[1])
for i in range(len(data.target_names)):
    bucket = df[df['Target'] == i]              # untuk mengambil data dengan target yang seragam
    bucket = bucket.iloc[:,[0,1]].values        # untuk mendapatkan aray berupa posisi x dan y, berukuran n x 2, n adalah banyak data
    hull = ConvexHull(bucket)                   # bagian ini diganti dengan hasil implementasi ConvexHull Divide & Conquer
    plt.scatter(bucket[:, 0], bucket[:, 1], label=data.target_names[i])
    for simplex in hull.simplices:
        plt.plot(bucket[simplex, 0], bucket[simplex, 1], colors[i])
plt.legend()

In [ ]:
bucket0 = df[df['Target'] == 0]
bucket0.head()

In [ ]:
bucket0 = bucket0.iloc[:,[0,1]].values
bucket0

In [ ]:
def DetFrom3Point(p1,p2,p3):
    mat = np.array([[1,1,1],
                    [p1[0],p2[0],p3[0]],
                    [p1[1],p2[1],p3[1]]])
    return np.linalg.det(mat)

def DistanceFromLine(p1,p2,p3):
    ''' Fungsi: Menerima 2 buah titik (p1 dan p2) yang membentuk segmen garis dan mengembalikan jarak dari p3 ke garis tsb
        Prekondisi : p1,p2,p3 adalah np.array 2 dimensi '''
    return np.cross(p2-p1,p3-p1)/np.linalg.norm(p2-p1)

def AngelFromLine(p1,p2,p3):
    ''' Fungsi: Menerima 2 buah titik (p1 dan p2) yang membentuk segmen garis p1p2 dan mengembalikan sudut p3p1p2
        Prekondisi : p1,p2,p3 adalah np.array 2 dimensi '''
    ba = p3 - p1
    bc = p2 - p1
    cosine = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    return np.arccos(cosine)

def NextSimplex(bucket,line,arr):             # UNTUK TAHAP KE-2 dst..
    ''' Proses: - Menerima himpunan titik pada suatu daerah yang ditandai oleh elemen array "arr" sebagai "indeks" dari array "bucket"
                - Array "bucket" memiliki elemen berupa titik secara universal
                - Array "line" memiliki elemen berupa "indeks" dari array "bucket" yang merupakan garis batas daerah sebelumnya
                - Fungsi akan mengembalikan simplex baru dari daerah tersebut
        Prekondisi : Daerah tersebut tidak kosong, minimal elemen array "arr" ada 1 '''
    
    ''' 1. Cari titik terjauh (p3) sehingga membentuk 2 buah line baru
           lineL adalah p1p3 berupa garis/ simplex sebelah KIRI
           lineR adalah p3p2 berupa garis/ simplex sebelah KANAN '''
    # a. Titik awal pada garis
    p1   = bucket[line[0]]
    Xone = bucket[line[0],0]
    Yone = bucket[line[0],1]
    
    # b. Titik akhir pada garis
    p2   = bucket[line[1]]
    Xend = bucket[line[1],0]
    Yend = bucket[line[1],1]
    
    d   = -1      # Jarak tidak mungkin negatif, inisiasi awal sebagai pembanding
    idx = -1      # Indeks ditemukannya titik dengan jarak terjauh   
    for i in arr:
        X = bucket[i,0]
        Y = bucket[i,1]
        p3 = np.array([[X,Y]])
        dTemp = DistanceFromLine(p1,p2,p3)
        if   (dTemp > d) :
            d   = dTemp
            idx = i
        elif ( dTemp==d ) :     # Jika jaraknya sama, makan bandingkan sudut
            dtheta    = AngelFromLine(p1,p2,p3)
            Temptheta = AngelFromLine(p1,p2,p3)
            if (Temptheta > dtheta) :
                d   = dTemp
                idx = i
        
    ''' 2. Setelah didapat titik terjauh, bentuk 2 buah line baru '''
    # GARIS BAGIAN KIRI
    lineL = [line[0],idx]
    XoneL = bucket[lineL[0],0]
    YoneL = bucket[lineL[0],1]
    XendL = bucket[lineL[1],0]
    YendL = bucket[lineL[1],1]
    # GARIS BAGIAN KANAN
    lineR = [idx,line[1]]
    XoneR = bucket[lineR[0],0]
    YoneR = bucket[lineR[0],1]
    XendR = bucket[lineR[1],0]
    YendR = bucket[lineR[1],1]
        
    ''' 3. Bagi kumpulan titik pada array "arr" menjadi 2 partisi, namun yg dipakai hanya partisi KIRI
        Apabila berada pada garis p1-p3, maka abaikan '''   
    arrLineL = []
    arrLineR = []
    ''' 3.1. Check di garis atau tidak untuk kedua garis batas baru, di garis (det=0) '''
    for i in arr:
        X = bucket[i,0]
        Y = bucket[i,1]
        ''' 3.1.a. Check untuk bagian kiri lineL,
            apabila (det>0) maka masukkan ke array selanjutnya untuk diperiksa '''
        matL = np.array([[1,1,1],
                         [XoneL,XendL,X],
                         [YoneL,YendL,Y]])
        detL = np.linalg.det(matL)
        if   (detL > 0) :
            arrLineL += [i]
        ''' 3.1.b. Check untuk bagian kiri lineR,
            apabila (det>0) maka masukkan ke array selanjutnya untuk diperiksa '''
        matL = np.array([[1,1,1],
                         [Xone,Xend,X],
                         [Yone,Yend,Y]])
        detL = np.linalg.det(matL)
        if   (detL > 0) :
            arrLineL += [i]

    ''' 3.4 CONQUER & COMBINE masing-masing sisi (INI TAHAP TERSULIT)'''
    newSimplex = []                         # {array simplex hanya menampung "pasangan" idx dari "adjecency" point}
    #  URUTAN CONQUER = KIRI -> KANAN
    # K-1 : Apabila kiri kosong
    if ( left==[] ) :
        # K-1.1 : Kanan kosong, maka simplex tetep
        if ( right==[] ) :
            newSimplex += [line]
        # K-1.2 : Kanan ada, maka simplex diubah menjadi bagian kanan
        else :
            newSimplex += [line]
            newSimplex += [NextSimplex()]
            
    # K-2 : Apabila kiri ada, maka disimpan simpex batas, dan diubah menjadi bagian sebelah kiri
    else :
        # K-2.1 : Kanan kosong, maka simplex diubah menjadi bagian kiri
        if ( right==[] ) :
            newSimplex += [line]
            newSimplex += [NextSimplex()]
        else :
            newSimplex += [NextSimplex()]
            newSimplex += [NextSimplex()]
            
    return newSimplex

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ TAHAP KE-1 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
''' 1. Urutkan semua titik berdasarkan X-axis and Y-axis '''
bucket = bucket0
bucket.sort(axis=1)
bucket.sort(axis=0)

''' 2. Buat p1-pn sebagai line batas awal '''
line = [0,len(bucket)-1]
# a. Titik awal pada garis
p1   = bucket[line[0]]
# b. Titik akhir pada garis
p2   = bucket[line[1]]

''' 3. Bagi kumpulan titik yang lain menjadi 2 partisi
       Apabila berada pada garis p1-pn, maka abaikan '''
left  = []
right = []
''' 3.1. Check di garis atau tidak, di garis (det=0) '''
for i in range(len(bucket)):
    X = bucket[i,0]
    Y = bucket[i,1]
    p3 = np.array([[X,Y]])
    det = DetFrom3Point(p1,p2,p3)
    ''' 3.2 Jika tidak digaris, bagi menjadi bagian KIRI (det>0) dan KANAN (det<0) '''
    if   (det > 0) :
        left  += [i]
    elif (det < 0) :
        right += [i]

''' 3.4 DIVIDE & CONQUER & COMBINE masing-masing sisi (INI TAHAP TERSULIT) '''
simplex = []                                        # {array simplex hanya menampung "pasangan" idx dari "adjecency" point}
#  URUTAN CONQUER = KIRI -> KANAN
# K-1 : Apabila kiri kosong
if ( left==[] ) :
    # K-1.1 : Kanan kosong, maka simplex tetep
    if ( right==[] ) :
        simplex += [line]
    # K-1.2 : Kanan ada, maka simplex diubah menjadi bagian kanan
    else :
        simplex += [line]
        simplex += [NextSimplex()]
        
# K-2 : Apabila kiri ada, maka disimpan simpex batas, dan diubah menjadi bagian sebelah kiri
else :
    # K-2.1 : Kanan kosong, maka simplex diubah menjadi bagian kiri
    if ( right==[] ) :
        simplex += [line]
        simplex += [NextSimplex()]
    else :
        simplex += [NextSimplex()]
        simplex += [NextSimplex()]

In [ ]:
hull0 = ConvexHull(bucket0)
hull0.simplices

In [ ]:
for simpul in hull0.simplices:
    print(simpul)
    break
simpul
bucket0[simpul,0]
# bucket0[simpul,1]

In [9]:
# bucket1 = df[df['Target'] == 1]
# bucket1.head()

In [10]:
# bucket1 = bucket1.iloc[:,[0,1]].values
# hull1 = ConvexHull(bucket1)
# hull1.simplices

In [ ]:
plt.plot([1,1],[1,2],'b')
plt.plot([1,2],[2,2],'r')
plt.plot([2,1],[2,1],'g')